To stop processes
```bash
pkill -9 -f pw.x
pkill -9 -f gipaw.x

```

In [ ]:
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

In [ ]:
from ase.visualize import view
view(crystal)

In [ ]:
import os
print(os.getcwd())

In [ ]:
crystal=read("../structures/KTU-183_2_auto.cif")
view(crystal)

In [ ]:
len(crystal)

In [ ]:
%%time
%load_ext autoreload
%autoreload 2

from runner import pw_runner
from ase.io import read, write

import time


scf_times = []
gipaw_times = []

for i in range(13,14):

    np_pw = i+1
    np_gipaw = i+1

    pw_params = {
        'prefix':'crystal', 
        'restart_mode' : 'from_scratch',
        'tstress':True, 
        'tprnfor':True, 
        'nosym':True, 
        'ecutwfc':50, 
        'kpts':(1, 1, 1),
        # 'kpts':None, 
        'ecutrho' : 500,
        'occupations' : 'smearing', 
        'smearing' : 'gauss', 
        'degauss' : 1.0e-2
    }

    tms = read("../structures/tms.xyz")
    # crystal=read("../structures/MIN-167-350K-CuCbPyz.cif")
    # crystal=read("../structures/HIK-143 293K-activated.cif")
    # crystal=read("../structures/HIK-143 MeOH.cif")
    # crystal=read("../structures/KTU-183_2_auto.cif")
    a = 20.0
    crystal = tms
    crystal.set_cell([a, a, a])
    crystal.set_cell([(a, 0, 0), (0, a, 0), (0, 0, a)])
    crystal.set_pbc(True)

    calc_dir = pw_runner.make_calc_dir()

    print("Starting QE/SCF calculation...")
    start_time = time.time()
    pw_runner.run_pw_scf(calc_dir, crystal, num_proc_pw=np_pw, pw_params=pw_params)
    scf_time = time.time() - start_time

    print("Starting gipaw calculation...")
    gipaw_out = pw_runner.run_gipaw(calc_dir, num_proc_gipaw=np_gipaw)
    gipaw_time = time.time() - scf_time - start_time

    chemical_shifts_iso, chemical_shifts_tensors = pw_runner.parse_gipaw_output(calc_dir + '/espresso_gipaw.pwo', num_atoms=len(crystal))

    crystal.info['chemical_shifts_iso'] = chemical_shifts_iso
    crystal.info['chemical_shifts_tensors'] = chemical_shifts_tensors

    write(calc_dir+"/crystal.xyz", crystal, format='extxyz')

    scf_times.append(scf_time)
    gipaw_times.append(gipaw_time)


# print("SCF time: ", scf_time)
# print("GIPAW time: ", gipaw_time)

import matplotlib.pyplot as plt
import numpy as np

x = np.arange(1, len(scf_times)+1, 1, dtype=int)
plt.plot(x, scf_times, label='SCF time')
plt.plot(x, gipaw_times, label='GIPAW time')
plt.xticks(x)
plt.xlabel('Num. of MPI processes')
plt.ylabel('Time, s.')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(1, len(scf_times)+1, 1, dtype=int)
plt.plot(x, scf_times_5, label='SCF time')
plt.plot(x, gipaw_times_5, label='GIPAW time')
plt.xticks(x)
plt.xlabel('Num. of MPI processes')
plt.ylabel('Time, s.')
plt.legend()
plt.show()

In [ ]:
x

In [ ]:
print(start_time, scf_time, gipaw_time-start_time)

In [ ]:
struct = read(calc_dir+"/crystal.xyz", format="extxyz")
print( struct.get_potential_energy() )
print( struct.get_forces() )
print( struct.info['chemical_shifts_iso'])
print( struct.info['chemical_shifts_tensors'])

In [ ]:
import psutil
 
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
os.chdir(head_dir)
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

view(tms)


In [ ]:
print( crystal.get_cell() )
print( crystal.get_pbc() )

crystal=read("test/MIN-167-350K-CuCbPyz.cif")
crystal.get_forces()

In [ ]:
view(crystal)